## Gridradar API
https://gridradar.net/de

In [ ]:
#!/usr/bin/python3
import requests
import json
import pprint
from lib.gridradar_api import *
from lib.basic_functions import *

In [ ]:
import config

#Gridradar.net API configuration
APItoken = config.APItoken
#InfluxDB configuration
influx_host =  config.influx_host
influx_port = config.influx_port # os.environ.get('INFLUX_PORT')
influx_username = config.influx_username
influx_password = config.influx_password
influx_database = config.influx_database
location = config.location 


https://api.gridradar.net/aggr_functions?token=4W01v742EaQ9iZj11B55vjaA4RyOtsNmu6M26EH09mFxlswbOSxA2X9bxuwZOBnpaYXtXyXf+xeQYl2LnQxev35WbGQVruURZteEpe52C0FVv08wXcb9Xm5f6Dvn/L3a


##  Metrics
**KEY**                            DESCRIPTION\
**- frequency**\
PMU selection, frequency 100ms, max period per query 1h\
**- frequency-mean-1m**\
PMU selection, 1m avg., max period per query 30d\
**- frequency-mean-1s**\
PMU selection, 1s avg., max period per query 12h\
**- frequency-ucte-median-1s**\
Median of the pmu 1 second mean frequencies\
**- net-time**\
Network time deviation of the UCTE power grid\
**- phase-angle**\
PMU selection, phase angle 100ms, max period per query 1h\
**- phase-angle-first-1m**\
PMU selection, first val. of minute, max period per query 30d\
**- phase-angle-first-1s**\
PMU selection, first val. of second, max period per query 12h\


columns	
0	"function"
1	"description"
data	
0	
0	"count"
1	"number of non-null values"
1	
0	"mean"
1	"arithmetic average of values"
2	
0	"median"
1	"middle value of the sorted list of values."
3	
0	"spread"
1	"difference between the minimum and maximum values"
4	
0	"stddev"
1	"the standard deviation of values."
5	
0	"sum"
1	"sum of values"
6	
0	"difference"
1	"difference between subsequent values"
7	
0	"min"
1	"lowest value"
8	
0	"max"
1	"greatest value"
9	
0	"first"
1	"the value with the oldest timestamp"
10	
0	"last"
1	"the value with the newest timestamp"

In [ ]:
# two different loops:
# query high_resolution data with short timerange every 30min
# query low_resolution data every 6hrs



### initialize InfluxDB Handler

In [ ]:
#InfluxDB configuration
import influxdb

influxdb_client = influxdb.InfluxDBClient(
            influx_host,
            influx_port,
            influx_username,
            influx_password,
            influx_database, 
            ssl = False,
            verify_ssl = True
        )

## get data from API with dynamic time ranges

In [ ]:
## Request and header parameters
url = 'https://api.gridradar.net/query'
token = config.APItoken#'<token>'

headers = {
    'Content-type': 'application/json', 
    'Authorization': 'Bearer '+token
}


In [ ]:
#dict of available metrics
metrics_dict = {
    "frequency-ucte-median-1s":{'account type': 'free', "aggr":"1s", "timerange":24, 'metric_id':1}, #'"1d"'
    "net-time":{'account type': 'free', "aggr":"1s", "timerange":24, 'metric_id':2},           #'"1d"'
    "frequency":{'account type': 'individual', "aggr":"100ms", "timerange":1, 'metric_id':3},        #'"1h"'
    "frequency-mean-1m":{'account type': 'individual', "aggr":"1m", "timerange":720, 'metric_id':4}, #'"30d"'
    "frequency-mean-1s":{'account type': 'individual', "aggr":"1s", "timerange":12, 'metric_id':5},  #'"12h"'
    "phase-angle":{'account type': 'individual', "aggr":"100ms", "timerange":1, 'metric_id':6},      #'"1h"'
    "phase-angle-first-1m":{'account type': 'individual', "aggr":"1m", "timerange":720, 'metric_id':7}, #'"30d"'
    "phase-angle-first-1m":{'account type': 'individual', "aggr":"1s", "timerange":12, 'metric_id':8} #'"12h"'
}

#metric = "frequency"
#settings = metrics_dict[metric]
#aggr = settings['aggr']
#timerange = settings['timerange']
#account_type = settings['account type']
#account_type


In [ ]:
for metric, settings in metrics_dict.items():
    ## define API query parameters
    if (settings['account type']=='individual'):
        request = metric_request_individualAPI(metric,metrics_dict,location)
    else:
        request = metric_request_freeAPI(metric,metrics_dict) 
    query_timer=0
    start = int(datetime.utcnow().timestamp() * 1000)          
    response = getdatafromapi(url,token,request)
    #print(response.status_code)  
    query_timer = int(datetime.utcnow().timestamp() * 1000) - start
    
    # ## Converting the JSON response string to a Python dictionary
    
    response_dict=None
    if response.status_code ==200:
        logging.info("gridradar-API - response for metric: %s, %s", metric,response.status_code)
        
        start = int(datetime.utcnow().timestamp() * 1000)
        response_dict = json.loads(response.content)
        out_influxDB = convert_dataset(metric,response_dict,location)
        ## Pretty print response dictionary
        #pprint.pprint(response_dict)
        influxdb_client.write_points(out_influxDB,database= influx_database,
                                    time_precision="ms",
                                    batch_size=10000,
                                    protocol='json')
        influxdb_timer = int(datetime.utcnow().timestamp() * 1000) - start   
        print('processing successfull: ', metric,response.status_code)
    else:
        logging.error("gridradar-API - response for metric: %s, %s", metric,response.status_code)
        print('processing failed: : ', metric,response.status_code)
        pprint.pprint(request)
        

    out_influxDB_APIdebug = gridradar_API_monitor(metric,response,location,query_timer,influxdb_timer)
    influxdb_client.write_points(out_influxDB_APIdebug,database= influx_database,
                                    time_precision="ms",
                                    batch_size=10000,
                                    protocol='json')
    